# Sistemas expertos con Python y Experta

## Introducción

### Filosofía

Nuestro objetivo es implementar una alternativa Python a CLIPS, lo más compatible posible. Con el objetivo de facilitar al programador CLIPS la transferencia de todos sus conocimientos a esta plataforma.


### Características

* Compatible con Python 3.
* Implementación pura de Python.
* Matcher basado en el algoritmo RETE.

### Diferencia entre CLIPS y Experta

1. CLIPS es un lenguaje de programación, Experta es una biblioteca de Python. Esto impone algunas limitaciones a las construcciones que podemos hacer (especialmente al LHS de una regla).
2. CLIPS está escrito en C, Experta en Python. Es de esperar un impacto notable en el rendimiento.
3. En CLIPS agregas hechos usando `assert`, en Python `assert` es una palabra clave, por lo que usamos `declare` en su lugar.

### Installation

El docente del módulo MIA ha preparado un contenedor docker en el que podemos ejecutar este NoteBook de jupyter y además lleva instalado experta y python 3.6.

## Lo básico

Un sistema experto es un programa capaz de emparejar un conjunto de hechos con un conjunto de reglas para esos hechos y ejecutar algunas acciones basadas en las reglas de coincidencia.

### Hechos

Los hechos son la unidad básica de información de Experta. Son utilizados por el sistema para razonar sobre el problema.

Enumeremos algunos datos sobre *Hechos*, entonces… *metahechos*;)

Necesitamos preparar el Notebook importando experta

In [1]:
!pip install git+https://github.com/openmotics/om-experta.git
from experta import *

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

ModuleNotFoundError: No module named 'experta'

1. La clase Fact es una subclase de dict.

In [2]:
f = Fact(a=1, b=2)
f['a']

1

2. Por lo tanto un Hecho no mantiene un orden interno de elementos.

In [3]:
Fact(a=1, b=2)  # Order is arbirary :O

Fact(a=1, b=2)

3. A diferencia de dict , puedes crear un Hecho sin claves (solo valores), y `Fact` creará un índice numérico para tus valores.

In [4]:
f = Fact('x', 'y', 'z')
f[0]

'x'

4. Puede mezclar valores autonuméricos con valores-clave, pero primero se deben declarar los autonuméricos:

In [5]:
f = Fact('x', 'y', 'z', a=1, b=2)
f[1]

'y'

In [6]:
f['b']

2

### Reglas

En Experta una regla es invocable.

Las reglas tienen dos componentes, LHS (lado izquierdo) y RHS (lado derecho).

* El LHS describe (usando patrones) las condiciones en las que la regla debe ejecutarse (o activarse).
* El RHS es el conjunto de acciones a realizar cuando se activa la regla.

Para que un Hecho coincida con un Patron, todas las restricciones del patrón deben ser **True** cuando el Hecho se evalúa con respecto a él.

In [7]:
class MyFact(Fact):
    pass

@Rule(MyFact())  # This is the LHS
def match_with_every_myfact():
    """This rule will match with every instance of `MyFact`."""
    # This is the RHS
    pass

@Rule(Fact('animal', family='felinae'))
def match_with_cats():
    """
    Match with every `Fact` which:

      * f[0] == 'animal'
      * f['family'] == 'felinae'

    """
    print("Meow!")

Puede utilizar operadores lógicos para expresar condiciones LHS complejas.

In [8]:
class User(Fact):
    pass

@Rule(
    AND(
        OR(User('admin'),
           User('root')),
        NOT(Fact('drop-privileges'))
    )
)
def the_user_has_power():
    """
    The user is a privileged one and we are not dropping privileges.

    """
    enable_superpowers()

#### Hechos vs Patrones

La diferencia entre hechos y patrones es pequeña. De hecho, los patrones son solo hechos que contienen elementos condicionales de patrón (Pattern Conditional Elements PCE) en lugar de datos regulares. Se utilizan únicamente en el LHS de una regla.

Si no proporciona el contenido de un patrón como **PCE**, Experta incluirá el valor en un Literal PCE automáticamente.

Además, no puede declarar ningún Hecho que contenga un PCE; si lo hace, recibirá una excepción.

In [10]:
ke = KnowledgeEngine()
ke.declare(Fact(L("hi")))

TypeError: Declared facts cannot contain conditional elements

### DefFacts (Hechos iniciales?)

La mayoría de las veces, los sistemas expertos necesitan que esté presente un conjunto de hechos para que el sistema funcione. Este es el propósito del decorador DefFacts.

In [10]:
@DefFacts()
def needed_data():
    yield Fact(best_color="red")
    yield Fact(best_body="medium")
    yield Fact(best_sweetness="dry")

Todos los DefFacts dentro de **KnowledgeEngine** se llamarán cada vez que se llame al método de `reset()`.

### Base de hechos (KnowledgeEngine)

Este es el lugar donde ocurre toda la magia.
El primer paso es crear una subclase y usar `Rule` para decorar sus métodos.
Después de eso, puede crear una instancia, completarlo con hechos y finalmente ejecutarlo.

In [11]:
from experta import *

class Greetings(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield Fact(action="greet")

    @Rule(Fact(action='greet'),
          NOT(Fact(name=W())))
    def ask_name(self):
        self.declare(Fact(name=input("What's your name? ")))

    @Rule(Fact(action='greet'),
          NOT(Fact(location=W())))
    def ask_location(self):
        self.declare(Fact(location=input("Where are you? ")))

    @Rule(Fact(action='greet'),
          Fact(name=MATCH.name),
          Fact(location=MATCH.location))
    def greet(self, name, location):
        print("Hi %s! How is the weather in %s?" % (name, location))

engine = Greetings()
engine.reset()  # Prepare the engine for the execution.
engine.run()  # Run it!

Where are you? 
What's your name? 
Hi ! How is the weather in ?


#### Manejo de hechos
Los siguientes métodos se utilizan para manipular el conjunto de hechos que conoce el motor.

##### declare

Agrega un nuevo hecho a la lista de hechos (la lista de hechos conocidos por el motor).

In [12]:
engine = KnowledgeEngine()
engine.reset()
engine.declare(Fact(score=5))
engine.facts

FactList([(0, InitialFact()), (1, Fact(score=5))])

>El mismo hecho no se puede declarar dos veces a menos que Facts.duplication esté establecido en True.

##### retract

Elimina un hecho existente de la lista de hechos.

>Tanto el índice como el hecho se pueden utilizar con `retract`.

In [13]:
engine.retract(1)
engine.facts

FactList([(0, InitialFact())])

##### modify

Retira algún hecho de la lista de hechos y declara uno nuevo con algunos cambios. Los cambios se pasan como argumentos.


In [14]:
engine.declare(Fact(color="red"))
engine.facts

FactList([(0, InitialFact()), (2, Fact(color='red'))])

In [15]:
engine.modify(engine.facts[2], color='yellow', blink=True)
engine.facts

FactList([(0, InitialFact()), (3, Fact(color='yellow', blink=True))])

##### duplicate

Agrega un hecho nuevo a la lista de hechos utilizando un hecho existente como plantilla y agregando algunas modificaciones.

In [16]:
engine.facts

FactList([(0, InitialFact()), (3, Fact(color='yellow', blink=True))])

In [17]:
engine.duplicate(engine.facts[3], color="orange", blink=False)
engine.facts

FactList([(0, InitialFact()),
          (3, Fact(color='yellow', blink=True)),
          (4, Fact(color='orange', blink=False))])

#### Procedimiento de ejecución del motor.

Este es el proceso habitual para ejecutar `knowledgeEngine`.
1. Por supuesto, se debe crear una instancia de la clase.
2. Se debe llamar al método de reinicio (`reset()`):
   * Esto declara el hecho especial `InitialFact`, necesario para que algunas reglas funcionen correctamente.
   * Declare todos los hechos obtenidos por los métodos decorados con `@DefFacts`.
3. Se debe llamar al método de ejecución (`run()`). Esto inicia el ciclo de ejecución.

#### Ciclo de ejecución

En un estilo de programación convencional, el programador define explícitamente el punto de inicio, el punto de finalización y la secuencia de operaciones. Con **Experta**, no es necesario definir el flujo del programa de forma tan explícita. El conocimiento (Reglas) y los datos (Hechos) se separan y **KnowledgeEngine** se utiliza para aplicar el conocimiento a los datos.

#### El ciclo de ejecución básico es el siguiente:

1. Si se ha alcanzado el límite de activación de la regla, se detiene la ejecución.
2. Se selecciona para su ejecución la regla superior de la agenda. Si no hay reglas en el orden del día, se detiene la ejecución.
3. Se ejecutan las acciones RHS de la regla seleccionada (se llama al método). Como resultado, las reglas pueden activarse o desactivarse. Las reglas activadas (aquellas reglas cuyas condiciones se cumplen actualmente) se colocan en la agenda. La ubicación en la agenda está determinada por la prominencia de la regla y la estrategia actual de resolución de conflictos. Las reglas desactivadas se eliminan de la agenda.

#### Diferencia entre DefFacts y declarar

Ambos se utilizan para declarar hechos en la instancia del motor, pero:
* `declare` agrega los hechos directamente a la memoria de trabajo.
* Los generadores declarados con `DefFacts` se llaman mediante el método reset y todos los hechos generados se agregan a la memoria de trabajo usando `declare`.

## Referencia

El siguiente diagrama muestra todos los componentes del sistema y las relaciones entre ellos.

![diagrama](assets/diagrama.png)

### `Rule` (*Regla*)

La regla es el método básico para componer patrones. Puede agregar tantos patrones o elementos condicionales como desee a una regla y se activará si todos ellos coinciden. Por lo tanto, se comporta como AND por defecto.

```python
@Rule(<pattern_1>@Rule(<pattern_1>`,
      <pattern_2>,
      ...
      <pattern_n>)
def _():
    pass
```


### `salience` (*Prioridad*)

Este valor, por defecto 0, determina la prioridad de la regla con respecto a las demás. Las reglas con mayor relevancia (salience) se activarán antes que las reglas con menor importancia.

*r1 tiene prioridad sobre r2*
```python
@Rule(salience=1)
def r1():
    pass

@Rule(salience=0)
def r2():
    pass
```

### Elementos condicionales: componer patrones
#### `AND`
`AND` crea un patrón compuesto que contiene todos los hechos pasados como argumentos. Todos los patrones pasados deben coincidir para que coincida el patrón compuesto.

*Coincide si se declaran dos hechos, uno que coincide con Fact(1) y otro que coincide con Fact(2)*

```python
@Rule(AND(Fact(1),
          Fact(2)))
def _():
    pass
```

#### `O`
`O` crea un patrón compuesto en el que cualquiera de los patrones dados hará que la regla coincida.

*Coincidencia si existe un hecho que coincida con Fact(1) y/o un hecho que coincida con Fact(2)*
```python
@Rule(OR(Fact(1),
         Fact(2)))
def _():
    pass
```

>Advertencia: Si coinciden varios hechos, la regla se activará varias veces, una por cada combinación válida de hechos coincidentes.

#### `NO`

Este elemento coincide si el patrón dado no coincide con ningún hecho o combinación de hechos. Por lo tanto, este elemento coincide con la ausencia del patrón dado.

*Coincide si ningún hecho coincide con el Hecho(1)*
```python
@Rule(NOT(Fact(1)))
def _():
    pass
```

#### `TEST` (*PRUEBA*)

Verifique el invocable recibido con los valores vinculados actuales. Si la ejecución devuelve *True*, la evaluación continuará y se detendrá en caso contrario.

*Coincidencia de todos los números a , b , c donde a > b > c*
```python
@Rule(Number(MATCH.a),
      Number(MATCH.b),
      TEST(lambda a, b: a > b),
      Number(MATCH.c),
      TEST(lambda b, c: b > c))
def _(a, b, c):
    pass
```
#### `EXISTS` (*EXISTE*)

Este elemento condicional recibe un patrón y coincide si uno o más hechos coinciden con este patrón. Esto coincidirá solo una vez mientras existan uno o más datos coincidentes y dejará de coincidir cuando no haya datos coincidentes.

Coincidirá una vez cuando existan uno o más colores
```python
@Rule(EXISTS(Color()))
def _():
    pass
```

#### `FORALL` (*PARA TODOS*)

El elemento condicional FORALL proporciona un mecanismo para determinar si un grupo de elementos condicionales especificados se cumple para cada aparición de otro elemento condicional especificado.

*Casará cuando para cada hecho de Estudiante existe un hecho de Lectura, Escritura y Aritmética con el mismo nombre.*
```python
@Rule(FORALL(Student(MATCH.name),
             Reading(MATCH.name),
             Writing(MATCH.name),
             Arithmetic(MATCH.name)))
def all_students_passed():
  ` pass
```
> Nota: Todas las variables vinculadas capturadas dentro de una cláusula FORALL no se pasarán como contexto al lado derecho de la regla.
> Nota: Cada vez que se activa la regla, el hecho coincidente es el hecho inicial (`InitialFact`).


### Field Constraints: *Restricciones de Campo* para ordenar

#### L (Literal Field Constraint) (*Restricción de campo literal*)

Este elemento realiza una coincidencia exacta con el valor dado. La coincidencia se realiza utilizando el operador de igualdad `==`.

*Coincide si el primer elemento es exactamente 3*
```python
@Rule(Fact(L(3)))
def _():
    pass
```
>Nota: Este es la restricción de campo predeterminado que se utiliza cuando no se proporciona ninguna restricción de campo como valor de patrón.

#### W (Wildcard Field Constraint) (*Restricción de campo comodín*)

Este elemento coincide con cualquier valor.

*Coincide si algún hecho se declara con la clave `mykey`.*
```python
@Rule(Fact(mykey=W()))
def _():
    pass
```
> Nota: Este elemento solo coincide si el elemento existe.

#### P (Predicate Field Constraint) (*Restricción de campo de predicado*)
La coincidencia de este elemento es el resultado de aplicar el valor exigible dado al valor extraído del hecho. Si el callable devuelve *True*, la restricción de campo coincidirá; en otro caso, la restricción de campo no coincidirá.

*Coincide si se declara algún hecho cuyo primer parámetro sea una instancia de `int`*
```python
@Rule(Fact(P(lambda x: isinstance(x, int))))
def _():
    pass
```

### Composición de Restricciones de campo: `&` , `|` y `~`

Todas las restricciones de campo se pueden componer juntas utilizando los operadores de composición `&` , `|` y `~`.

#### `ANDFC()` también conocido como `&`

La restricción de campo compuesto se cumple si todas las restricciones de campo dadas se cumplen.

*Coincide si la clave x del Punto es un valor entre 0 y 255.*
```python
@Rule(Fact(x=P(lambda x: x >= 0) & P(lambda x: x <= 255)))
def _():
    pass
```

#### `ORFC()` también conocido como `|`

La restricción de campo compuesto se cumple si alguna de las restricciones de campo se cumple.

*Coincide si el nombre es Alice o Bob .*
```python
@Rule(Fact(name=L('Alice') | L('Bob')))
def _():
    pass
```

#### `NOTFC()` también conocido como `~`

Esta restricción de campo compuesto niega la restricción de campo dada, invirtiendo la lógica. Si la restricción de campo se cumple, este no lo hará y viceversa.

*Coincide si el nombre no es Charlie.*
```python
@Rule(Fact(name=~L('Charlie')))
def _():
    pass
```

### Enlace de variables: el operador `<<`

Cualquier patrón y algunas restricciones de campo se pueden vincular a un nombre utilizando el operador `<<`.

*El primer valor del hecho coincidente se vinculará al nombre `value` y se pasará a la función cuando se active.*
```python
@Rule(Fact('value' << W()))
def _(value):
    pass
```

### objeto `MATCH` (*COINCIDIR*)

Los objetos MATCH ayudan a generar enlaces de nombres más legibles. Es azúcar sintáctico para una restricción de campo comodín vinculada a un nombre. Por ejemplo:
```python
@Rule(Fact(MATCH.myvalue))
def _(myvalue):
    pass
```
Es exactamente lo mismo que:
```python
@Rule(Fact("myvalue" << W()))
def _(myvalue):
    pass
```
### objeto `AS` (*COMO*)

El objeto `AS`, como el objeto `MATCH`, es azúcar sintáctico para generar nombres enlazables. En este caso, cualquier atributo solicitado al objeto `AS` devolverá una cadena con el mismo nombre.

```python
@Rule(AS.myfact << Fact(W()))
def _(myfact):
    pass
```
Es exactamente lo mismo que:
```python
@Rule("myfact" << Fact(W()))
def _(myfact):
    pass
```
> Advertencia: Este comportamiento variará en futuras versiones de Experta y el tipo de cadena del operador puede desaparecer.

### Coincidencia anidada

Nuevo en la versión 1.3.0.

La coincidencia anidada es útil para comparar valores de hechos que contienen estructuras anidadas como dictados o listas.
Las coincidencias anidadas toman la forma campo__subclave=valor. (Eso es un doble guión bajo). Por ejemplo:

In [12]:
from experta import *

class Nested(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield Fact(name="scissors", against={"scissors": 0, "rock": -1, "paper": 1})
        yield Fact(name="paper", against={"scissors": -1, "rock": 1, "paper": 0})
        yield Fact(name="rock", against={"scissors": 1, "rock": 0, "paper": -1})

    @Rule(Fact(name=MATCH.name, against__scissors=1, against__paper=-1))
    def what_wins_to_scissors_and_losses_to_paper(self, name):
        print(name)

engine = Nested()
engine.reset()  # Prepare the engine for the execution.
engine.run()  # Run it!    

rock


Es posible compararlo con una estructura profunda arbitraria siguiendo el mismo método.
En este ejemplo podemos comprobar la colisión entre un barco y su padre con la siguiente regla:

In [31]:
from experta import *

class Ship(KnowledgeEngine):      
    @DefFacts()
    def _initial_action(self):
        yield Fact(data={
                    "name": "SmallShip",
                    "position": {
                        "x": 200,
                        "y": 300},
                    "parent": {
                        "name": "BigShip",
                        "position": {
                            "x": 150,
                            "y": 300}}})

    @Rule(Fact(data__name=MATCH.name1,
        data__position__x=MATCH.x,
        data__position__y=MATCH.y,
        data__parent__name=MATCH.name2,
        data__parent__position__x=MATCH.x,
        data__parent__position__y=MATCH.y))
    def collision_detected(self, name1, name2, **_):
        print("COLLISION!", name1, name2)
    
engine = Ship()
engine.reset()  # Prepare the engine for the execution.
engine.run()  # Run it!    

> Prueba a cambiar las posiciones de los dos barcos anteriores (para que coincidan) y vuelve a ejecutar el código.

Si la estructura de datos anidada contiene listas, tuplas o cualquier otra secuencia, puede utilizar índices numéricos según sea necesario.

In [33]:
from experta import *

class Ship(KnowledgeEngine):      
    @DefFacts()
    def _initial_action(self):
        yield Fact(data={
                    "name": "SmallShip",
                    "position": {
                        "x": 200,
                        "y": 300},
                    "enemies": [
                    {"name": "Destroyer"},
                    {"name": "BigShip"}]})

    @Rule(Fact(data__enemies__0__name="Destroyer"))
    def next_enemy_is_destroyer(self):
         print("Bye byee!")
    
engine = Ship()
engine.reset()  # Prepare the engine for the execution.
engine.run()  # Run it!    

Bye byee!


### Objetos mutables

El algoritmo de comparación de Experta depende de que los valores de los hechos declarados sean inmutables.

Cuando se crea un hecho, todos sus valores se transforman a un tipo inmutable si no es así. Para esto se utiliza internamente el método `experta.utils.freeze`.

In [34]:
class MutableTest(KnowledgeEngine):
     @Rule(Fact(v1=MATCH.v1, v2=MATCH.v2, v3=MATCH.v3))
     def is_immutable(self, v1, v2, v3):
         print(type(v1), "is Immutable!")
         print(type(v2), "is Immutable!")
         print(type(v3), "is Immutable!")

ke = MutableTest()
ke.reset()
ke.declare(Fact(v1={"a": 1, "b": 2}, v2=[1, 2, 3], v3={1, 2, 3}))
ke.run()

<class 'frozendict.frozendict'> is Immutable!
<class 'experta.utils.frozenlist'> is Immutable!
<class 'frozenset'> is Immutable!


> Nota: Puede importar `frozendict` y `frozenlist` desde el módulo `experta.utils`. Sin embargo, `frozenset` es un tipo integrado en Python.

#### Registre su propio congelador mutable

Si necesita incluir sus propios tipos mutables personalizados como valores de hecho, debe registrar un congelador de tipo especializado para su tipo personalizado.

```python
from experta.utils import freeze
@freeze.register(MyType)
def freeze_mytype(obj):
    return ... # My frozen version of my type
```

#### Descongelar objetos congelados

Para descongelar fácilmente los objetos congelados, `experta.utils` contiene un método de descongelación .

In [37]:
from experta import *
from experta.utils import *

class MutableTest(KnowledgeEngine):
     @Rule(Fact(v1=MATCH.v1, v2=MATCH.v2, v3=MATCH.v3))
     def is_immutable(self, v1, v2, v3):
         print(type(unfreeze(v1)), "is Mutable!")
         print(type(unfreeze(v2)), "is Mutable!")
         print(type(unfreeze(v3)), "is Mutable!")

ke = MutableTest()
ke.reset()
ke.declare(Fact(v1={"a": 1, "b": 2}, v2=[1, 2, 3], v3={1, 2, 3}))
ke.run()

<class 'dict'> is Mutable!
<class 'list'> is Mutable!
<class 'set'> is Mutable!


> Nota: El mismo procedimiento de registro de congelación que se muestra arriba también se aplica al descongelamiento.

# Fuentes de información
- https://experta.readthedocs.io/en/latest/introduction.html
- https://clipsrules.net/documentation/v624/ug624.pdf